In [16]:
#Use findspark to confirm pyspark works
import findspark
import streamlit
findspark.init()

In [ ]:
#Import pyspark
import pyspark
from pyspark import SparkContext, SparkConf
from collections import Counter
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import split, col

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

conf = SparkConf().setAppName('Project').setMaster('local')
sparkContext = SparkContext.getOrCreate(conf=conf) #SparkContext(conf = conf)

In [45]:
# Read in and assign column names
# Income data
Income = spark.read.csv('income_csv.csv',header=False)
Income = Income.toDF('Area_code','Area','Self_num_ind','Self_mean','Self_median','Num_ind','Mean','Median','Pens_num_ind','Pens_mean',
                     'Pens_median','Other_num_ind','Other_mean','Other_median','Tot_num_ind','Tot_mean','Tot_median','Tax_num_ind','Tax_mean','Tax_median','Tot_tax')

# Drop not applicable
Income = Income.na.drop()

# Remove original formatting
Income = Income.filter(Income.Area!='Region/County')

# Population data
Population = spark.read.csv('population_data.csv',header=False)
Population = Population.toDF('Code','Name','Geography','All ages','0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15',
                     '16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','37',
                     '38','39','40','41','42','43','44','45','46','47','48','49','50','51','52','53','54','55','56','57','58','59',
                     '60','61','62','63','64','65','66','67','68','69','70','71','72','73','74','75','76','77','78','79','80','81','82','83','84','85','86','87','88','89','90+')

# Drop not applicable
Population = Population.na.drop()

# Remove original formatting
Population = Population.filter(Population.Code!='Code')

# Income data
Statements = spark.read.csv('Transposed_Consolidated_Income_Statements.csv',header=False)
Statements = Statements.toDF('Company','Fina_period','State','Revenue','Cost','Gross','Other','Disp_close','Admin','Operating','Fin_out','Fin_in','Share_loss','Before_tax','Tax','Profit')

# Remove original formatting
Statements = Statements.drop(col('Company'))
Statements = Statements.drop(col('Fina_period'))
Statements = Statements.drop(col('State'))

# Remove original formatting
Statements = Statements.filter(Statements.Cost!='Cost of sales')

Statements.show()

+-------+------+-----+-----+----------+-----+---------+-------+------+----------+----------+----+------+
|Revenue|  Cost|Gross|Other|Disp_close|Admin|Operating|Fin_out|Fin_in|Share_loss|Before_tax| Tax|Profit|
+-------+------+-----+-----+----------+-----+---------+-------+------+----------+----------+----+------+
|  18358|-17857|  501|  114|         0| -308|      307|    -77|     7|        -1|       236| -34|   202|
|      0|   -26|  -26|    0|         1| -193|     -218|      0|    33|         0|      -185|  35|  -150|
|  18358|-17883|  475|  114|         1| -501|       89|    -77|    40|        -1|        51|   1|    52|
|  18479|-18075|  404|  123|         0| -281|      246|    -67|     3|        -1|       181| -33|   148|
|      0|   -13|  -13|    0|        10| -225|     -228|    -21|    30|         0|      -219|  41|  -178|
|  18479|-18088|  391|  123|        10| -506|       18|    -88|    33|        -1|       -38|   8|   -30|
|  65762|-61005| 4690| 1634|         0|-2060|     2630|